
### **Project IA Machine Learning**


In [1]:
!pip install langdetect

In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn  import svm
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from langdetect import detect
from sklearn.svm import SVC
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from langdetect import LangDetectException

### **Analyse des données**
Nous avons commencé en analysant le jeu de données fourni, définir quelles informations seraient utiles à notre recherche, ainsi que les manières de nettoyer ces données.

In [3]:
data = pd.read_csv("text_sentiment.csv", sep=',', usecols=range(4))

Ce code permet de lire le jeu de données au format .csv

In [4]:
data.head(10)

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
5,1956968477,worry,xxxPEACHESxxx,Re-pinging @ghostridah14: why didn't you go to...
6,1956968487,sadness,ShansBee,"I should be sleep, but im not! thinking about ..."
7,1956968636,worry,mcsleazy,Hmmm. http://www.djhero.com/ is down
8,1956969035,sadness,nic0lepaula,@charviray Charlene my love. I miss you
9,1956969172,sadness,Ingenue_Em,@kelcouch I'm sorry at least it's Friday?


In [5]:
sentiment = data["sentiment"]
sentiment
sentiment.unique()

array(['empty', 'sadness', 'enthusiasm', 'neutral', 'worry', 'surprise',
       'love', 'fun', 'hate', 'happiness', 'boredom', 'relief', 'anger'],
      dtype=object)

Nous analysons ici la donnée sentiment, et nous faisons en sorte d'en éliminer les doublons avec .unique()

### **Nettoyage des données**
Nous passons maintenant au nettoyage des données, en commençant par retirer les ID et les ID des tweets qui sont des informations inutiles

In [6]:
dataclean = data.drop(['tweet_id', 'author'], axis=1)
dataclean

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


### **Suppression des élements doublons**
Nous supprimons les données en double pour éviter de fausser l'entraînement de l'IA

In [7]:
dataclean = dataclean.drop_duplicates()
dataclean

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


### **Suppression des données vides**
Nous supprimons les données vies, pour éviter de fausser l'entraînement de l'IA

In [8]:
data_filtered = dataclean.dropna()
data_filtered

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


### **Ajout de la fonction pour supprimer les mots qui commencent par @**
Les mentions n'apportant rien à l'information du message, nous nous en débarrassons, nous supprimons donc l'arobase et le mot qui la suit.

In [9]:
def remove_arobase(text):
  """
  This function removes words starting with "@" from a string.
  """
  return " ".join(word for word in text.split() if not word.startswith("@"))

### **Suppression des @**

In [10]:
data_filtered["content"] = data_filtered["content"].apply(remove_arobase)

In [11]:
data_filtered

,sentiment,content
0,empty,i know i was listenin to bad habit earlier and...
1,sadness,Layin n bed with a headache ughhhh...waitin on...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,We want to trade with someone who has Houston ...
...,...,...
39995,neutral,
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,WASSUP BEAUTIFUL!!! FOLLOW ME!! PEEP OUT MY NE...


Maintenant il faut s'assurer que le contenu est dans la même langue, toujours pour ne pas fausser l'entraînement. Au vu du jeu de données, garder les tweets en anglais permettra d'avoir un maximum de données à analyser

### **Suppression des lignes que ne sont pas en anglais**

In [12]:
# def test_filter(text):
#     try:
#         return detect(text) == "en"
#     except LangDetectException:
#         return False

# data_filtered["content"].apply(lambda text: test_filter(text))

In [13]:
# Filter for English content (boolean Series)
english_pattern=r"^[A-Z].*\."
english_mask = data_filtered["content"].apply(lambda text: bool(re.fullmatch(english_pattern, text)))

df_filtered = data_filtered[english_mask].copy()
df_filtered

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin on...
2,sadness,Funeral ceremony...gloomy friday...
4,neutral,We want to trade with someone who has Houston ...
17,sadness,So sleepy again and it's not even that late. I...
29,worry,I HATE CANCER. I HATE IT I HATE IT I HATE IT.
...,...,...
39911,happiness,WOOHOO!!! My broom isnt up to standards....the...
39919,neutral,Nearly there...
39961,happiness,"Wow! Up, coffee in hand and already outside. S..."
39984,surprise,Oh that's why.


### **Ajout de la fonction pour supprimer les #**

Les #, qui précèdent les hashtags dans un tweet, risquent de fausser l'entraînement, néanmoins, contrairement aux mentions, les hashtags indiquent une information de contexte dans le contexte de l'entraînement de l'IA. Il faut juste supprimer le symbole.

In [1]:
def remove_hashtags(text):
  """
  Cette fonction supprime les "#" d'une chaîne de caractères.
  """
  return text.replace("#", "")

### **Suppression des #**

In [15]:
df_filtered["content"] = df_filtered["content"].apply(remove_hashtags)

In [16]:
df_filtered

,sentiment,content
1,sadness,Layin n bed with a headache ughhhh...waitin on...
2,sadness,Funeral ceremony...gloomy friday...
4,neutral,We want to trade with someone who has Houston ...
17,sadness,So sleepy again and it's not even that late. I...
29,worry,I HATE CANCER. I HATE IT I HATE IT I HATE IT.
...,...,...
39911,happiness,WOOHOO!!! My broom isnt up to standards....the...
39919,neutral,Nearly there...
39961,happiness,"Wow! Up, coffee in hand and already outside. S..."
39984,surprise,Oh that's why.


### **Analyse des labels aprés nettoyage de dataset (nombre de type sentiments)**

Nous devons nous arrêter, après ce nettoyage au préalable pour analyser les sentiments restants

In [17]:
sentiment = df_filtered["sentiment"]
sentiment
sentiment.unique()
print(sentiment.nunique())

13


### **Occurrences de chaque sentiment**

Nous analysons le nombre de sentiments exprimés, pour pouvoir s'assurer d'un échantillonnage équilibré

In [18]:
sentiment_counts = df_filtered["sentiment"].value_counts()
print(sentiment_counts)

worry         1268
neutral       1023
sadness        784
happiness      499
love           396
surprise       254
relief         211
hate           178
fun            173
empty           86
enthusiasm      82
boredom         25
anger           16
Name: sentiment, dtype: int64


### **Suppression des sentiments avec une occurrence inférieure à *1000*

Pour pouvoir entraîner l'IA de la meilleure manière, nous retirons tous les sentiments ayant un nombre d'occurrences inférieur à 1000

In [2]:
# def test_filter(text):
#     try:
#         return detect(text) == "en"
#     except LangDetectException:
#         return False

# data_filtered["content"].apply(lambda text: test_filter(text))

In [20]:
sentiments_a_supprimer = sentiment_counts[sentiment_counts < 1000].index.to_list()
data2 = df_filtered[~df_filtered["sentiment"].isin(sentiments_a_supprimer)]
sentiment_counts = data2["sentiment"].value_counts()
print(sentiment_counts)

worry      1268
neutral    1023
Name: sentiment, dtype: int64


### **Mise à niveau de tous les sentiments à 1000**

Il est nécessaire de tronquer les exemples pour atteindre un nombre identique dans le jeu de données, pour ne pas créer de biais artificiel vers une des émotions et fausser les résultats

In [3]:
nombre_occurrences = 1000
# sentiment_counts = data2["sentiment"].value_counts()
# sentiments_to_keep = sentiment_counts[sentiment_counts > 1000].index.to_list()

# data2 = data2[data2["sentiment"].isin(sentiments_to_keep)]

data_extrait = pd.DataFrame()
for sentiment in data2["sentiment"].unique():
    # Filtrer les lignes du sentiment actuel
    sentiment_data = data2[data2["sentiment"] == sentiment]

    # Extraire les 1000 premières lignes
    data_extrait = data_extrait.append(sentiment_data.head(nombre_occurrences))


data_extrait["sentiment"].value_counts()
print(data_extrait["sentiment"].value_counts())

NameError: name 'pd' is not defined


### **Fonctions sur la compréhension de la donnée après nettoyage**



### **Tokenisition des stopWords et définition de fonction de filtre**

In [22]:
nltk.download('stopwords')
en_stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()


def tokenize_and_filter(text):
    result = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', result)
    text2 = [ps.stem(word) for word in tokens if word not in en_stopwords]
    return text2


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### **Vectorisation de la donnée**

In [23]:
vectorisation_full = TfidfVectorizer(analyzer=tokenize_and_filter)
vect_final = vectorisation_full.fit_transform(data2['content'])

In [24]:
vect_final

<2291x4709 sparse matrix of type '<class 'numpy.float64'>'
	with 19536 stored elements in Compressed Sparse Row format>

In [25]:
vect_final.shape

(2291, 4709)

### **Train test-split**

In [26]:
# X_train, X_test, y_train, y_test = train_test_split(vect_final, data2["sentiment"], test_size=0.2, random_state=42)

In [27]:
# y_train

### **Encodage du label**

In [28]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(data2["sentiment"])

In [29]:
X_train, X_test, y_train, y_test = train_test_split(vect_final.toarray(),
                                                     y_encoded, test_size=0.2, random_state=42)


### **Création du model avec fit et Support Vector Classifier**

In [30]:
# model = SVC(kernel='linear')
# model.fit(X_train, y_train_encoded)

### **Définition d'un modèle avec réseaux de neurones**



In [31]:
model = Sequential([
    Embedding(input_dim=vect_final.shape[1], output_dim=128),  # Embedding layer for TF-IDF vectors
    LSTM(units=64, return_sequences=False),  # LSTM layer
    Dense(units=32, activation='relu'),  # MLP layer
    Dense(units=len(set(y_train)), activation='softmax')  # Output layer for sentiment classes
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         602752    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 654306 (2.50 MB)
Trainable params: 654306 (2.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [33]:
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))


Epoch 1/10
15/15 [==============================] - 7s 287ms/step - loss: 0.6893 - accuracy: 0.5551 - val_loss: 0.6888 - val_accuracy: 0.5468
Epoch 2/10
15/15 [==============================] - 4s 294ms/step - loss: 0.6878 - accuracy: 0.5551 - val_loss: 0.6890 - val_accuracy: 0.5468
Epoch 3/10
15/15 [==============================] - 4s 269ms/step - loss: 0.6872 - accuracy: 0.5551 - val_loss: 0.6888 - val_accuracy: 0.5468
Epoch 4/10
15/15 [==============================] - 6s 398ms/step - loss: 0.6875 - accuracy: 0.5551 - val_loss: 0.6890 - val_accuracy: 0.5468
Epoch 5/10
15/15 [==============================] - 4s 282ms/step - loss: 0.6871 - accuracy: 0.5551 - val_loss: 0.6888 - val_accuracy: 0.5468
Epoch 6/10
15/15 [==============================] - 3s 193ms/step - loss: 0.6872 - accuracy: 0.5551 - val_loss: 0.6890 - val_accuracy: 0.5468
Epoch 7/10
15/15 [==============================] - 3s 221ms/step - loss: 0.6874 - accuracy: 0.5551 - val_loss: 0.6891 - val_accuracy: 0.5468
Epoch 

In [34]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)

15/15 [==============================] - 1s 77ms/step - loss: 0.6895 - accuracy: 0.5468
Test Accuracy: 0.5468409657478333
